## Fetch 2009 PhysioNet challenge data

https://archive.physionet.org/challenge/2009/

In [ ]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import requests
import tarfile
import wfdb

from src.data.parsing import parse_txt

source_url = 'https://archive.physionet.org/challenge/2009/training-set-clinical-data.tar.gz'
target_dir = 'data'
target_path = f'{target_dir}/training-set-clinical-data.tar.gz'




In [ ]:
def set_config():
    return {
        'fetch_clinical_data': False
    }

## Fetch archive of PhysioNet challenge records

In [ ]:
if set_config()['fetch_clinical_data']:
    response = requests.get(source_url, stream=True)
    if response.status_code == 200:
        with open(target_path, 'wb') as f:
            f.write(response.raw.read())

    with tarfile.open(target_path, "r:gz") as tar_file:
        tar_file.extractall(target_dir)
else: 
    print("Use cached clinical data")

## Handle individual records

In [ ]:
record_map = pd.read_csv(
    'data/mimic2cdb/MAP', 
    sep="\t", 
    names = ['Clinical', 'Wave', 'Sex', 'Age', 'Birthdate', 'Waveform'],
    index_col = False, 
    skiprows = [0,1])
record_map.head()


## Try using WFDB

In [ ]:
record = wfdb.rdrecord(f"data/train_wave/{record_map['Wave'][0]}")
display(record.__dict__)
df = pd.DataFrame(record.p_signal, columns = record.sig_name)
df = df[~df['ABP'].isna()]
print(df[['ABP']].plot())